In [9]:
import gzip

In [10]:
dataset = []

f = gzip.open("steam_games.json.gz")
for l in f:
    d = eval(l)
    dataset.append(d)
        
f.close()

In [11]:
len(dataset)

32135

In [14]:
#first game
dataset[61]

{'publisher': 'CINEMAX, s.r.o.',
 'genres': ['Casual', 'Indie'],
 'app_name': 'Gumboy - Crazy Adventures™',
 'sentiment': 'Mixed',
 'title': 'Gumboy - Crazy Adventures™',
 'url': 'http://store.steampowered.com/app/2520/Gumboy__Crazy_Adventures/',
 'release_date': '2006-12-19',
 'tags': ['Casual', 'Indie', 'Singleplayer'],
 'reviews_url': 'http://steamcommunity.com/app/2520/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'metascore': 69,
 'early_access': False,
 'id': '2520',
 'developer': 'CINEMAX, s.r.o.'}

In [4]:
# Print all keys in the first record
print("Keys in the first record:")
print(dataset[0].keys())

Keys in the first record:
dict_keys(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date', 'tags', 'discount_price', 'reviews_url', 'specs', 'price', 'early_access', 'id', 'developer'])


In [15]:
sentiments = [game['sentiment'] for game in dataset if 'sentiment' in game]
unique_sentiments = set(sentiments)

print("Unique sentiment values:")
print(unique_sentiments)

Unique sentiment values:
{'Overwhelmingly Negative', '9 user reviews', 'Mostly Positive', 'Very Negative', '6 user reviews', 'Mixed', '7 user reviews', 'Negative', 'Mostly Negative', 'Positive', 'Very Positive', '5 user reviews', 'Overwhelmingly Positive', '4 user reviews', '2 user reviews', '8 user reviews', '1 user reviews', '3 user reviews'}
